In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.4.1"
# spark_version = 'spark-3.<enter version>'
os.environ["SPARK_VERSION"]=spark_version
# Verify the name of the hadoop2 TGZ file into the spark version folder, eg: spark-3.3.1-bin-hadoop2.tgz
hadoop_version = "hadoop3"
os.environ["HADOOP_VERSION"] = "hadoop3"
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-{hadoop_version}"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [850 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,103 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

In [2]:
# Import findspark and initialize.

# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [5]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home_sales')


In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg (price),2) AS Average_Price
             FROM home_sales
             WHERE bedrooms==4
             group By date_built
             order by date_built""").show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg (price),2) AS Average_Price
             FROM home_sales
             WHERE bedrooms==3
             AND bathrooms==3
             group By date_built
             order by date_built""").show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [8]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""SELECT date_built, round(avg (price),2) AS Average_Price
             FROM home_sales
             WHERE bedrooms==3
             AND bathrooms==3
             AND floors==2
             AND sqft_lot >= 2000
             group By date_built
             order by date_built
             """).show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    280447.23|
|      2011|    281413.45|
|      2012|    295858.68|
|      2013|    295142.13|
|      2014|    294195.13|
|      2015|    291788.36|
|      2016|     287812.6|
|      2017|    282026.59|
+----------+-------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT round(avg (view),2)
             FROM home_sales
             WHERE price >=350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 1.9681787490844727 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT round(avg (view),2)
             AS Average_View_Rating
             FROM home_sales
             WHERE price >=350000""").show()



print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+
|Average_View_Rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.7917726039886475 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 11. Read the formatted parquet data.
home_df_2 = spark.read.parquet('home_sales_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
home_df_2.createOrReplaceTempView('home_sales_2')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""SELECT round(avg (view),2)
             AS Average_View_Rating
             FROM home_sales_2
             WHERE price >=350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|Average_View_Rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.7484769821166992 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False